In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.feature_selection import SequentialFeatureSelector, RFE, mutual_info_classif, SelectKBest, f_classif, chi2
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, ElasticNet
from feature_selection_package.evaluation import performance_score, single_evaluation, full_evaluation
from feature_selection_package.feature_selectors import CorrelationSelector, MutualInformationSelector, RandomForestSelector, EnsembleSelector
from boruta import BorutaPy
from feature_selection_package.sms_data_creation import get_word_counts_train_test

# Load dataset

In [3]:
# Load data sms
sms_train = pd.read_csv('data/sms_train.csv')
sms_train_data, sms_train_labels = sms_train.iloc[:, 1], sms_train.iloc[:, 0]

sms_test_data = pd.read_csv('data/sms_test.csv')
sms_test = sms_test_data.copy()
sms_test['label'] = np.nan

preprocessed_sms_train , preprocessed_sms_test = get_word_counts_train_test(sms_train, sms_test)


In [5]:
# oversample
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
preprocessed_sms_train, sms_train_labels = oversample.fit_resample(preprocessed_sms_train, sms_train_labels)    


In [6]:
train_data = preprocessed_sms_train.iloc[:np.round(len(preprocessed_sms_train)*0.8).astype(int), 1:]
train_labels = preprocessed_sms_train.iloc[:np.round(len(preprocessed_sms_train)*0.8).astype(int), 0]
valid_data = preprocessed_sms_train.iloc[np.round(len(preprocessed_sms_train)*0.8).astype(int):, 1:]
valid_labels = preprocessed_sms_train.iloc[np.round(len(preprocessed_sms_train)*0.8).astype(int):, 0]

# Check if train and valid data is balanced

In [4]:
print('number of observations in each class in train set:', np.unique(train_labels, return_counts=True))
print('number of observations in each class in validation set:', np.unique(valid_labels, return_counts=True))

number of observations in each class in train set: (array([0, 1], dtype=int64), array([3170,  488], dtype=int64))
number of observations in each class in validation set: (array([0, 1], dtype=int64), array([789, 125], dtype=int64))


# Estimators

In [5]:
svm1 = SVC(kernel='rbf', C=1, random_state=0)
svm2 = SVC(kernel='linear', C=1, random_state=0)

tree = DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=0)
xgboost = XGBClassifier(learning_rate=0.1, n_estimators=100, max_depth=5, random_state=0)
rfc = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=0)

logreg = LogisticRegression(penalty='l2', C=1, random_state=0, max_iter=1000)

classifiers = np.array([svm1, svm2, tree, xgboost, rfc, logreg])

# Feature selection methods

## Dimensionality reduction methods

### PCA

In [6]:
n_features = [10, 20, 30, 50, 75]
pca_results = pd.DataFrame()
for n in n_features:
    selector = [PCA(n_components=n)]
    pca_df = full_evaluation(train_data, train_labels, valid_data, valid_labels, selector, classifiers, dataset_type='sms')
    pca_results = pd.concat([pca_results, pca_df])
pca_results.to_csv('data/pca.csv', index=False)

In [7]:
pd.read_csv('data/pca.csv')

,Selector,Classifier,Number_of_Features,Accuracy,Performance_score
0,PCA,SVC_rbf,10,0.912198,0.9122
1,PCA,SVC_linear,10,0.894099,0.8941
2,PCA,DecisionTreeClassifier,10,0.929029,0.9290
3,PCA,XGBClassifier,10,0.941465,0.9415
4,PCA,RandomForestClassifier,10,0.901465,0.9015
5,PCA,LogisticRegression,10,0.870733,0.8707
6,PCA,SVC_rbf,20,0.906099,0.9061
7,PCA,SVC_linear,20,0.898733,0.8987
8,PCA,DecisionTreeClassifier,20,0.941663,0.9417
9,PCA,XGBClassifier,20,0.937465,0.9375


## Wrapper methods

### RFE

In [ ]:
n_features = [10, 50, 100]
selector = [RFE(estimator=RandomForestClassifier(n_estimators=100, max_depth=3), n_features_to_select=7000, step=1, verbose=0)]
rfe_results = full_evaluation(train_data, train_labels, valid_data, valid_labels, selector, classifiers, dataset_type='sms')
rfe_results.to_csv('data/RFE.csv', index=False)

## Embeded methods

### Lasso

In [9]:
lasso_df = pd.DataFrame(columns=['Selector', 'Classifier', 'Number_of_Features', 'Accuracy', 'Performance_score'])
Cs = [0.0001, 0.001, 0.01, 0.1, 1, 10]
for C in Cs:
    lasso = LogisticRegression(penalty='l1', C=C, solver='liblinear', random_state=0)
    lasso.fit(train_data, train_labels)
    n_features = sum(lasso.coef_[0] != 0)
    score = lasso.score(valid_data, valid_labels)
    perf_score = performance_score(score, n_features, dataset_type='sms')
    lasso_df = pd.concat([lasso_df, pd.DataFrame({'Selector': ['Lasso'], 'Classifier': ['Lasso'], 'Number_of_Features': [n_features], 'Accuracy': [score], 'Performance_score': [perf_score]})], ignore_index=True)

lasso_df.to_csv('data/lasso.csv', index=False)

### Elastic net

In [71]:
alphas = [0.0001, 0.001, 0.01, 0.1, 1, 10]
l1_ratio = [0.9, 0.95, 0.98]
elastic_df = pd.DataFrame(columns=['Selector', 'Classifier', 'Number_of_Features', 'Accuracy', 'Performance_score', 'alpha', 'l1_ratio'])
for ratio in l1_ratio:
    for alpha in alphas:
        elastic = ElasticNet(alpha=alpha, l1_ratio=ratio, random_state=0, max_iter = 10000)
        elastic.fit(train_data, train_labels)
        n_features = sum(elastic.coef_!= 0)
        score = elastic.score(valid_data, valid_labels)
        perf_score = performance_score(score, n_features, dataset_type='sms')
        elastic_df = pd.concat([elastic_df, pd.DataFrame({'Selector': ['Lasso'], 'Classifier': ['Lasso'], 'Number_of_Features': [n_features], 'Accuracy': [score], 'Performance_score': [perf_score], "alpha": [alpha], "l1_ratio": [ratio]})], ignore_index=True)

elastic_df.to_csv('data/elasticNet.csv', index=False)

In [72]:
elasticNet = pd.read_csv('data2/elasticNet.csv')
elasticNet['Selector'] = 'ElasticNet'
elasticNet['Classifier'] = 'ElasticNet'
elasticNet.to_csv('data/elasticNet.csv', index=False)

### Random forest

In [ ]:
n_features = [10, 20, 30, 50, 75, 100, 200]
forest_results = pd.DataFrame()
for n in n_features:
    selector = [RandomForestSelector(n_features=n)]
    df = full_evaluation(train_data, train_labels, valid_data, valid_labels, selector, classifiers, dataset_type='sms')
    forest_results = pd.concat([forest_results, df])
forest_results.to_csv('data/forest.csv', index=False)

## Filter methods

### Correlation coefficient

In [16]:
n_features = [10, 20, 30, 50, 75, 100, 200]
corr_results = pd.DataFrame()
for n in n_features:
    selector = [CorrelationSelector(n_features=n)]
    df = full_evaluation(train_data, train_labels, valid_data, valid_labels, selector, classifiers, dataset_type='sms')
    corr_results = pd.concat([corr_results, df])
corr_results.to_csv('data/corr.csv', index=False)

### Mutual information

In [ ]:
n_features = [10, 20, 30, 50, 75, 100, 200]
mutual_results = pd.DataFrame()
for n in n_features:
    selector = [MutualInformationSelector(n_features=n)]
    df = full_evaluation(train_data, train_labels, valid_data, valid_labels, selector, classifiers, dataset_type='sms')
    mutual_results = pd.concat([mutual_results, df])
mutual_results.to_csv('data/mutual.csv', index=False)

### Select K - Best

#### ANOVA

In [ ]:
n_features = [10, 20, 30, 50, 75, 100, 200]
anova_results = pd.DataFrame()
for n in n_features:
    selector = [SelectKBest(f_classif, k=n)]
    df = full_evaluation(train_data, train_labels, valid_data, valid_labels, selector, classifiers, dataset_type='sms')
    anova_results = pd.concat([anova_results, df])
anova_results.to_csv('data/anova.csv', index=False)

### chi2

In [ ]:
n_features = [10, 20, 30, 50, 75, 100, 200]
chi2_results = pd.DataFrame()
for n in n_features:
    selector = [SelectKBest(chi2, k=n)]
    df = full_evaluation(train_data, train_labels, valid_data, valid_labels, selector, classifiers, dataset_type='sms')
    chi2_results = pd.concat([chi2_results, df])
chi2_results.to_csv('data/chi2.csv', index=False)

## Hybrid + wrapper

### Boruta algorithm

In [ ]:
selector = [BorutaPy(estimator=RandomForestClassifier(n_estimators=100, max_depth=3), n_estimators='auto', verbose=1, random_state=0)]
boruta_results = full_evaluation(train_data, train_labels, valid_data, valid_labels, selector, classifiers, dataset_type='sms')
boruta_results.to_csv('data/boruta.csv', index=False)

## Stacking

In [7]:
n_features_rfs = [100, 200]
n_features_rfe = [10, 25, 50]

In [9]:
stack_results = pd.DataFrame()
for n in n_features_rfs:
    for m in n_features_rfe:
        selector1 = RandomForestSelector(n_features=n)
        selector2 = RFE(estimator=RandomForestClassifier(n_estimators=100, max_depth=3), n_features_to_select=m, step=1, verbose=1)
        selectors = [[selector1, selector2]]
        df = full_evaluation(train_data, train_labels, valid_data, valid_labels, selectors, classifiers, dataset_type='artificial')
        stack_results = pd.concat([stack_results, df])
        stack_results.to_csv('data/stack.csv', index=False)

stack_results.to_csv('data/stack.csv', index=False)

Fitting estimator with 100 features.
Fitting estimator with 99 features.
Fitting estimator with 98 features.
Fitting estimator with 97 features.
Fitting estimator with 96 features.
Fitting estimator with 95 features.
Fitting estimator with 94 features.
Fitting estimator with 93 features.
Fitting estimator with 92 features.
Fitting estimator with 91 features.
Fitting estimator with 90 features.
Fitting estimator with 89 features.
Fitting estimator with 88 features.
Fitting estimator with 87 features.
Fitting estimator with 86 features.
Fitting estimator with 85 features.
Fitting estimator with 84 features.
Fitting estimator with 83 features.
Fitting estimator with 82 features.
Fitting estimator with 81 features.
Fitting estimator with 80 features.
Fitting estimator with 79 features.
Fitting estimator with 78 features.
Fitting estimator with 77 features.
Fitting estimator with 76 features.


KeyboardInterrupt: 

## Ensemble

In [ ]:
n_features = [10, 20, 30, 50, 75, 100, 200]
ensemble_results = pd.DataFrame()
for n in n_features:
    selectors = [RandomForestSelector(n_features=n), SelectKBest(f_classif, k=n), SelectKBest(chi2, k=n), CorrelationSelector(n_features=n), MutualInformationSelector(n_features=n)]
    ensemble = [EnsembleSelector(selectors=selectors)]
    df = full_evaluation(train_data, train_labels, valid_data, valid_labels, ensemble, classifiers, dataset_type='sms')
    ensemble_results = pd.concat([ensemble_results, df])
    
ensemble_results.to_csv('data/ensemble.csv', index=False)